# Salary Value For NBA Players: Combining Salary with Advanced Stats to Determine Ultimate Player Worth & Grade GM Transactions

### Background

Determining the quality of a basketball team's makeup has been the focus of sports data analysis for several years. Advanced stats, measuring all facets of a player's game, from win shares, PIPM, PER, etc each have made their mark in the analytics space.  However, examining the process of constructing a team is something that certainly warrants further analysis and comprehension.  Specifically, the degree to which contract dollars and player performance are correlated appears to be an area that, analytically, is untapped (at least publically).  

The ability of an NBA general manager (GM) to proficiently perform their job has long been defined, perhaps too simplistically, by wins and losses on the court.  However, I believe that the minutiae of the individual transactions that a GM performs is more indicative of their overall basketball acumen vs the wins and losses that I believe are more indicative of coaching.  As fans, we love to dissect the moves that a GM makes.  How many times have you read about a transaction that your favorite team just made, or saw a Woj bomb and thought “Man, if only I ran the team!” Maybe you’ve had the feeling that a talented team fell on some bad luck or that a particular team was full of overpaid players. Maybe you’ve thought that a botched trade doomed a team for years or that posturing for cap space is an ulimately flawed endeavour. How a GM pays for talent is behind every team’s success or lack thereof.  

Rather than simply say "Masai Ujiri was the best GM of the 2018-19 season because the Toronto Raptors won the NBA title" - I am looking to analyze and grade the transactions that Ujiri has made each season that led his team to the title.  I'd like to put a quantitative value on particular transactions rather than say "It was a good trade" or "He should have never signed that guy."  

I sought out to find a way to analyze, rate, and compare GMs over the years based on a number of factors.  The primary ways in which a GM can affect their team's wins and losses are via free agent signings, the draft, trades, and releasing players.  With each of these transactions, the common theme is that the particular success or or failure of said transaction is tied to the player's performance, the capital needed to acquire that player, and the degree to which the transaction allows the franchise to remain financially flexible.  

To grade a player and/or his contract, I decided to take a look at two main data points: 1) each team’s yearly cash expenditure on every player they employed, and 2) the stats that those players accumulated while receiving that cash. I believe that this derived ‘net value’ lays the groundwork to answer several potential followup questions, which I'll theorize at the conclusion of this post - the first in a series dedicated to the guys who build, modify, bring greatness to, and ultimately doom our favorite teams.

### Methodology

Each player contract and/or season is calculated to have an "contract adjusted value" (CAV) and a "net value" (NET).  CAV estimates what a player's contract "should be" considering their advanced stats.  NET is the difference between a player's actual contract and their CAV value.  NET scales the dollar amount of a contract to the average dollar amount of a contract for a player performing at a similar level according to whatever "all-in-one" advanced analytic statistical value is chosen.  This is meant to differentiate a player that has a VORP of 4.9 making $10,000,000/yr vs another with the same VORP making only $2,500,000/yr.  The 'average' NET would be zero; that is to say, an average NBA player would have a NET of zero because he is playing at exactly his contract's worth.    

For the purposes of grading historic seasons, the player's actual values are used. For future seasons, the player's predicted stats are used, calculated using k-nearest neighbors and machine learning.  This matches a player to other similar historical players, by previous stats at the same ages, and then looks at the trajectory of their stats going forward in order to "predict" our player's stats. 

I looked for a table that had player’s advanced stats, draft position, and contract info all in once place. This proved impossible (surprisingly, given the data world’s passion for all things roundball-related), so I decided to make gather and wrangle my own data.

I scraped data from both basketball reference and spotrac. These both involved quite amount of cleaning (BR had inconsistent salaries and spotrac had inconsistent player cash take-homes prior to 2015-16, just to name a few issues). Traded players also required cash adjustments. A minor note: player performance (and thus GM performance) was compared to a player’s cash take home (what a GM spent on a player) and not necessarily their cap hit, which is the traditional measure of a team’s spend. A player traded mid season would not have a cap hit for their initial team, but obviously they accumulate stats and receive a salary - which for purposes of this analysis, should be the basis of their value. Also noted in the scrape was any outcome/note for a particular contract - this included ten-day, waived, two-way, and amnestied contracts.

Defining 'how a player performs' is a bit more subjective.  Traditional box score stats generally don't translate well across different teams/situations (eg - points per game doesn't mean much if a player's usage rate is unusually high on a team that isn't winning much, or if they simply just shoot the ball a lot).  They also don't give a complete scope of how a player is performing - a player may be scoring a lot, but how is he on defense?  

All-in-one/advanced stats (PER, WS, BPM/VORP, TS%, etc) have been in the lexicon now for years, but there is no consensus on which one to use.  They can be dependent on a number of variables - teammates, roles, coaching, offensive style, rotations, roster makeup.  I was looking for a stat which best represented the portability of a player's talent - ie what stat would best represent a player's abilities if he went to another team? 

I decided to do some research; Ben Taylor at Nylon Calculus had a great write-up on advanced stats (https://fansided.com/2019/01/08/nylon-calculus-best-advanced-stat/).  This article really dives into the weeds - while there is no definitive conclusion, it makes a good case for a few stats, including box plus/minus (BPM), which can be scraped from basketball reference.  The only drawback is that BPM is a rate statistic and will exclude a large portion of players, making it impossible to rate entire teams.  I would need something that allows for every player to be rated.  VORP - which is based on BPM, but adjusts for minutes played - would work well for this.  Basketball reference has this statistic, so I decided to go with VORP for this project. 

### Toggle the Raw Code On/Off If You'd Like

In [4]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [106]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup, Comment
import requests
import time
import re
import math
from datetime import date
from datetime import datetime

from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_squared_error
from math import sqrt

import warnings
warnings.filterwarnings('ignore')

## 2019-2020 Advanced Stats Scrape 

For this introductory post, I wanted to take a look at the final numbers of the 2019-20 NBA regular season to see which players had the most 'efficient' seasons (Future posts will look into both historical seasons and future season).  Of course, I'm looking at how they performed vs. how they were paid - thus combining the efficiency of both the player and his GM.  

Scraping the player, age, team, and BPM/VORP from the 2019-20 NBA regular season on basketball reference (Note: 'Season' column is from the second half; ie: 2019-2020 would be listed as 2020): 

In [107]:
startTime = datetime.now()

#all teams 1980-current
all_league = ['BOS','NYK','TOR','NJN','BRK','PHI','CHI','DET','IND','MIL','CLE','ATL','MIA','WAS','CHA','CHO','CHH','ORL','LAL','LAC','GSW','PHO','POR','UTA','DEN','MEM','VAN','OKC','SEA','NOH','NOP','DAL','HOU','SAS','MIN','SAC']

#current teams
league = ['BOS','NYK','TOR','BRK','PHI','CHI','DET','IND','MIL','CLE','ATL','MIA','WAS','CHO','ORL','LAL','LAC','GSW','PHO','POR','UTA','DEN','MEM','OKC','NOP','DAL','HOU','SAS','MIN','SAC']

years = map(str, range(1990, 2021))

players = []
season = []
teams = []
ages = []
games = []
bpm = []
vorp = []

for year in years:
    for team in all_league:
        html = 'https://www.basketball-reference.com/teams/'+team+'/'+year+'.html'
        webpage = requests.get(html)
        content = webpage.content
        soup = BeautifulSoup(re.sub("<!--|-->","", content.decode('utf-8')),'lxml')
        adv = soup.find('table',{'id' : 'advanced'})
        if adv == None:
            pass
        else:
            for item in adv.find_all('tr'):
                for value in item.find_all('td',{'data-stat':'player'}):
                    player = value.text
                    players.append(player)
                    teams.append(team)
                    season.append(year)
                for value in item.find_all('td',{'data-stat':'g'}):
                    game = value.text
                    games.append(game)
                for value in item.find_all('td',{'data-stat':'age'}):
                    age = value.text
                    ages.append(age)
                for value in item.find_all('td',{'data-stat':'bpm'}):
                    bp = value.text
                    bpm.append(bp)
                for value in item.find_all('td',{'data-stat':'vorp'}):
                    v = value.text
                    vorp.append(v)
                
nba = pd.DataFrame({'Player':players,'Age':ages,'Year':season,'Team':teams,'G':games,'BPM':bpm, 'VORP':vorp})
nba.head(), print(datetime.now() - startTime)

0:14:16.108224


(           Player Age  Year Team   G   BPM VORP
 0      Larry Bird  33  1990  BOS  75   5.3  5.5
 1    Kevin McHale  32  1990  BOS  82   3.4  3.8
 2    Reggie Lewis  24  1990  BOS  79   0.9  1.9
 3   Robert Parish  36  1990  BOS  79   1.1  1.9
 4  Dennis Johnson  35  1990  BOS  75  -0.3  0.9, None)

Determining a player's first year:

In [108]:
players = set(list(nba['Player']))

dfss = []

for p in players:
    a = nba[nba['Player']==p]
    min_yr = min(a['Year'])
    a['First Year'] = min_yr  
    dfss.append(a)
  
nba3 = pd.concat(dfss)

nba3.head()

,Player,Age,Year,Team,G,BPM,VORP,First Year
11573,Nate Wolters,22,2014,MIL,58,-1.8,0.1,2014
12130,Nate Wolters,23,2015,MIL,11,-4.6,-0.1,2014
12433,Nate Wolters,23,2015,NOP,10,-7.5,-0.1,2014
14029,Nate Wolters,26,2018,UTA,5,-12.8,-0.1,2014
226,Joe Wolf,25,1990,LAC,77,-5.3,-1.1,1990


## Adding in Draft Pick/Year via Scrape 

Scraping draft pick status from basketball reference: 

In [159]:
seasons = list(range(1950,2020))

name = []
ids = []
picks = []
years = []
for year in seasons:

    html = 'https://www.basketball-reference.com/draft/NBA_'+str(year)+'.html'
    webpage = requests.get(html)
    content = webpage.content
    soup = BeautifulSoup(content,'lxml')
    adv = soup.find('table',{'id' : 'stats'})
    for item in adv.find_all('tr'):
        for value in item.find_all('td',{'data-stat':'player'}):
            player = value.text
            name.append(player)
            player_str = player.split()
            player_id = (player_str[1]+player_str[0]+str(year))
            ids.append(player_id)
        for value in item.find_all('td',{'data-stat':'pick_overall'}):
            pick = value.text
            picks.append(pick)
            years.append(year)
            
draft = pd.DataFrame({'Player':name,'ID':ids,'Overall Pick':picks,'Season':years})
draft_dict = dict(zip(name,picks))
draft_year = dict(zip(name,years))

draft.head()

,Player,ID,Overall Pick,Season
0,Paul Arizin,ArizinPaul1950,,1950
1,Chuck Share,ShareChuck1950,1,1950
2,Don Rehfeldt,RehfeldtDon1950,2,1950
3,Bob Cousy,CousyBob1950,3,1950
4,Dick Schnittker,SchnittkerDick1950,4,1950


## Scraping Salary 

Scraping salary from basketball reference:

In [10]:
league = ['BOS','NYK','TOR','PHI','BRK','CHI','DET','IND','MIL','CLE','ATL','MIA','WAS','CHO','ORL','LAL','LAC','GSW','PHO','SAC','POR','UTA','DEN','MIN','OKC','NOP','DAL','HOU','SAS','MEM']

yr = 2020

players = []
seasons = []
teams = []
salaries = []
signed_using = []
notes = []


for team in league:
    
    player_number = 0
    
    html = 'https://www.basketball-reference.com/contracts/'+team+'.html'
    webpage = requests.get(html)
    content = webpage.content
    soup = BeautifulSoup(content,'lxml')
    adva = soup.find('table',{'id' : 'contracts'})
    for item in soup.find_all('tr'):

        for value in item.find_all('td',{'data-stat':'y1'}):
            salary = value.text
            if len(salary)<12:
                salaries.append(salary)

        for value in item.find_all('td',{'data-stat':'signed_using'}):
            sign = value.text
            signed_using.append(sign)
            
        for value in item.find_all('th',{'data-stat':'player','class':'right'}):
            play = value.text
            players.append(play)
        
        for value in item.find_all(href=True):
            p = value.text
            players.append(p)
            seasons.append(yr)
            teams.append(team)
            
            comments = soup.find_all(string=lambda text: isinstance(text, Comment))
            payroll = comments[22]
            pay = pd.read_html(str(payroll))[0]
            pn = pay['Notes'][player_number]
            notes.append(pn)
            player_number += 1
    
    del signed_using[-1]       

salaries20 = pd.DataFrame({'Player':players,'Team':teams,'Salary':salaries, 'Signed Using':signed_using, 'Notes':notes})

salaries20.head()

,Player,Team,Salary,Signed Using,Notes
0,Kemba Walker,BOS,"$32,742,000",Sign and Trade,2022-23 is player option. Signed 4-yr $140.79M...
1,Gordon Hayward,BOS,"$32,700,690",Cap space,2020-21 is a player option. Signed 4-yr/$128M ...
2,Marcus Smart,BOS,"$12,553,471",Bird Rights,"Signed 4-yr/$52MM contract on July 19, 2018."
3,Jayson Tatum,BOS,"$7,830,000",1st Round Pick,"2020-21 team option exercised October 14, 2019..."
4,Jaylen Brown,BOS,"$6,534,829",1st Round pick,Signed 4-yr $107M contract extension on Octobe...


Combining salaries with advanced stats and draft picks: 

In [160]:
nba3['Year'] = nba3['Year'].astype(int)

nba20 = nba3[nba3['Year']==2020]

new_nba = pd.merge(nba20, salaries20, on=['Player', 'Team'], how='outer')

new_nba['Year'] = new_nba['Year'].fillna(2020)
new_nba['Year'] = new_nba['Year'].astype(int)

new_nba['G'] = new_nba['G'].fillna(0)
new_nba['G'] = new_nba['G'].astype(int)
new_nba['BPM']= pd.to_numeric(new_nba['BPM'], errors='coerce')
new_nba = new_nba.replace(np.nan, 0, regex=True)

new_nba["Draft Pick"] = new_nba['Player'].map(draft_dict)
new_nba["Draft Year"] = new_nba['Player'].map(draft_year)

new_nba['Draft Pick'] = new_nba['Draft Pick'].fillna('Undrafted')
new_nba['Draft Year'] = new_nba['Draft Year'].fillna('Undrafted')

new_nba['Salary'] = new_nba['Salary'].replace(u'\xa0', u' ')
new_nba['Salary'] = new_nba['Salary'].str.replace(' ','')
new_nba['Salary'] = new_nba['Salary'].str.replace('$','')
new_nba['Salary'] = new_nba['Salary'].str.replace(',','')
new_nba['Salary'] = new_nba['Salary'].fillna(0)

new_nba['Salary'] = new_nba['Salary'].replace('',0)
new_nba['Salary'] = new_nba['Salary'].astype(int)

new_nba['G'] = new_nba['G'].fillna(0)
new_nba['G'] = new_nba['G'].astype(int)

new_nba['BPM'] = new_nba['BPM'].astype(float)
new_nba['BPM'] = new_nba['BPM'].fillna(0)

new_nba['First Year'] = new_nba['First Year'].astype(int)

cols1 = ['Player','Age','Year','Team','G','BPM','VORP','Draft Pick','Draft Year','Salary','Signed Using','Notes','First Year']

new_nba = new_nba[cols1]

df = []
players = set(list(new_nba['Player']))
for p in players:
    n = new_nba[new_nba['Player']==p]
    fy = n['First Year'].max()
    if fy == 0:
        o = nba3[nba3['Player']==p]
        first = o['First Year'].max()
        n['First Year'] = first
    else:
        n['First Year'] = fy
    df.append(n)
    
nba4 = pd.concat(df)

nba4['First Year'] = nba4['First Year'].replace(0,2020)
nba4['First Year'] = nba4['First Year'].fillna(2020)
    
nba4.head()

,Player,Age,Year,Team,G,BPM,VORP,Draft Pick,Draft Year,Salary,Signed Using,Notes,First Year
300,Will Barton,29,2020,DEN,58,1.1,1.5,40,2012,12960000,Bird Rights,2021-2022 is player option. Signed 4-yr/$54MM ...,2013
448,Iman Shumpert,29,2020,BRK,13,-5.5,-0.2,17,2011,419443,,"Waived December 12, 2019. Signed 1-yr minimum ...",2012
159,Harry Giles,21,2020,SAC,46,-1.3,0.1,20,2017,2578800,1st Round Pick,"2020-21 team option declined October 31, 2019....",2019
622,Cameron Reynolds,0,2020,MIL,0,0.0,0,Undrafted,Undrafted,0,Two-Way Contract,"Signed two-way contract July 26, 2019.",2019
160,De'Anthony Melton,21,2020,MEM,60,-0.2,0.5,46,2018,1416852,Cap Space,"Traded from PHO to MEM July 7, 2019. Signed 2-...",2019


In [161]:
wally = nba4[nba4['Player']=='John Wall']
wally

,Player,Age,Year,Team,G,BPM,VORP,Draft Pick,Draft Year,Salary,Signed Using,Notes,First Year
640,John Wall,0,2020,WAS,0,0.0,0,1,2010,37800000,Bird Rights,15% trade kicker. 2022-23 is player option. Si...,2011


Creating team-specific salaries for players who were traded or who changed teams during the season.  Traditionally, the last team to end up with a player will be assigned his salary cap figure for the year.  However if, for example, 50% of a player's games are with one team, and 50% with another, his stats should be tied to the salary he recieved for that specific team, so that he can be assigned a value that he provided for that calculated salary.  This is shown in the "new salary" column below (note, the 'new salary' of a a player who stayed with one team all year will be the same as the 'salary' column):  

In [162]:
nba_players = set(list(nba4['Player']))

dfs = []

cols = ['Player','Age','Year','Team','G','BPM','VORP','Draft Pick','Draft Year','Salary','Signed Using','Notes','First Year','Prorated Salary','Combined?']

for p in nba_players:
    a = nba4[nba4['Player']==p]
    if len(a)==1:
        a['Prorated Salary'] = a['Salary']
        a['Combined?'] = 'No'
        dfs.append(a)
    else:
        teams = set(list(a['Team']))
        sal_sum = a['Salary'].sum()
        if len(teams) > 1:
            sals = list(a['Salary'])
            if 0 in sals:
                games = a['G'].sum()
                for i, row in a.iterrows():
                    games_ratio = a['G']/games
                    a['Prorated Salary'] = sal_sum*games_ratio
                    a['Combined?'] = 'No'
                dfs.append(a)
            else:
                a['Prorated Salary'] = a['Salary']
                a['Combined?'] = 'No'
                dfs.append(a)

        else:
            age = a['Age'].values[0]
            year = a['Year'].values[0]
            team = a['Team'].values[0]
            g = a['G'].values[0]
            bpm = a['BPM'].values[0]
            vorp = a['VORP'].values[0]
            dp = a['Draft Pick'].values[0]
            dy = a['Draft Year'].values[0]
            salary = 0
            pr_salary = a['Salary'].sum()
            signed = set(list(a['Signed Using']))
            combined = 'Yes'
            notes = set(list(a['Notes']))
            fy = a['First Year'].values[0]
                
            b = pd.DataFrame(np.array([[p],[age],[year],[team],[g],[bpm],[vorp],[dp],[dy],[salary],[signed],[notes],[fy],[pr_salary],[combined]]).T,columns=cols) 
            dfs.append(b)

#'combined' would only equal yes if player was with only one team, but with two different stints in the same year  
            
nba5 = pd.concat(dfs)

nba6 = nba5[cols]
nba6.head()

,Player,Age,Year,Team,G,BPM,VORP,Draft Pick,Draft Year,Salary,Signed Using,Notes,First Year,Prorated Salary,Combined?
300,Will Barton,29,2020,DEN,58,1.1,1.5,40,2012,12960000,Bird Rights,2021-2022 is player option. Signed 4-yr/$54MM ...,2013,12960000,No
448,Iman Shumpert,29,2020,BRK,13,-5.5,-0.2,17,2011,419443,,"Waived December 12, 2019. Signed 1-yr minimum ...",2012,419443,No
159,Harry Giles,21,2020,SAC,46,-1.3,0.1,20,2017,2578800,1st Round Pick,"2020-21 team option declined October 31, 2019....",2019,2578800,No
622,Cameron Reynolds,0,2020,MIL,0,0,0,Undrafted,Undrafted,0,Two-Way Contract,"Signed two-way contract July 26, 2019.",2019,0,No
160,De'Anthony Melton,21,2020,MEM,60,-0.2,0.5,46,2018,1416852,Cap Space,"Traded from PHO to MEM July 7, 2019. Signed 2-...",2019,1416852,No


## Creating the 'Net Value' (NET) & 'Contract Adjust Value' (CAV) Categories

Finally, creating Net Value and Contract Adjusted Value categories which gives a value to a player based on his VORP and salary received; table is sorted by NET: 

In [163]:
nba7 = nba6

total_salary = nba7['Salary'].sum()

nba7['VORP'] = nba7['VORP'].fillna(0)

nba7['VORP'] = nba7['VORP'].astype(float)
total_vorp = nba7['VORP'].sum()
cost_per_vorp = total_salary/total_vorp

nba7['CAV'] = (nba7['VORP']*cost_per_vorp)

nba7['NET Value'] = (nba7['VORP']*cost_per_vorp) - nba7['Prorated Salary']

nba7['Year'] = nba7['Year'].fillna(2020)
nba7['First Year'] = nba7['First Year'].fillna(2020)

nba7['Year'] = nba7['Year'].astype(int)
nba7['First Year'] = nba7['First Year'].astype(int)

nba7.loc[nba7['Player'] == 'Jaren Jackson' , 'Draft Pick'] = 4
nba7.loc[nba7['Player'] == 'Tim Hardaway' , 'Draft Pick'] = 24
nba7.loc[nba7['Player'] == 'Larry Nance' , 'Draft Pick'] = 27
nba7.loc[nba7['Player'] == 'Gary Payton' , 'Draft Pick'] = 'Undrafted'
nba7.loc[nba7['Player'] == 'Glenn Robinson' , 'Draft Pick'] = 40
nba7.loc[nba7['Player'] == 'Gary Trent' , 'Draft Year'] = 37

nba7.loc[nba7['Player'] == 'Jaren Jackson' , 'Draft Year'] = 2018
nba7.loc[nba7['Player'] == 'Tim Hardaway' , 'Draft Year'] = 2013
nba7.loc[nba7['Player'] == 'Larry Nance' , 'Draft Year'] = 2015
nba7.loc[nba7['Player'] == 'Gary Payton' , 'Draft Year'] = 'Undrafted'
nba7.loc[nba7['Player'] == 'Glenn Robinson' , 'Draft Year'] = 2014
nba7.loc[nba7['Player'] == 'Gary Trent' , 'Draft Year'] = 2018

nba7.loc[nba7['Player'] == 'Jaren Jackson' , 'First Year'] = 2019
nba7.loc[nba7['Player'] == 'Tim Hardaway' , 'First Year'] = 2014
nba7.loc[nba7['Player'] == 'Larry Nance' , 'First Year'] = 2016
nba7.loc[nba7['Player'] == 'Gary Payton' , 'First Year'] = 2017
nba7.loc[nba7['Player'] == 'Glenn Robinson' , 'First Year'] = 2015
nba7.loc[nba7['Player'] == 'Gary Trent' , 'First Year'] = 2019

nba7['Season Num'] = ((nba7['Year']) - (nba7['First Year']))+1

nba7.to_csv('/Users/atrain31/Desktop/NBA Project/Stats/nba_2020_net_value.csv')

nba7.sort_values('NET Value', ascending=False)

,Player,Age,Year,Team,G,BPM,VORP,Draft Pick,Draft Year,Salary,Signed Using,Notes,First Year,Prorated Salary,Combined?,CAV,NET Value,Season Num
116,Luka Dončić,20,2020,DAL,61,8.4,5.4,3,2018,7683360,1st Round Pick,"2020-21 team option exercised September 24, 20...",2019,7683360,No,7.926291e+07,7.15795e+07,2
302,Giannis Antetokounmpo,25,2020,MIL,63,11.5,6.6,15,2013,25842697,1st Round Pick,"Signed 4-yr/$100M extension on September 20, 2...",2014,25842697,No,9.687689e+07,7.10342e+07,7
529,James Harden,30,2020,HOU,68,9.6,7.3,3,2009,37800000,Bird Rights,"Signed extension thru 2022-23 on July 8, 2017....",2010,37800000,No,1.071517e+08,6.93517e+07,11
146,Damian Lillard,29,2020,POR,66,7.5,5.9,6,2012,29802321,1st Round Pick,"Signed a 4-year contract extension on July 6, ...",2013,29802321,No,8.660207e+07,5.67997e+07,8
79,Nikola Jokić,24,2020,DEN,73,7.4,5.5,41,2014,26573595,Bird Rights,"Signed 5-yr maximum salary contract on July 9,...",2016,26573595,No,8.073074e+07,5.41571e+07,5
120,Anthony Davis,26,2020,LAL,62,8,5.4,1,2012,27093019,1st Round Pick,"Traded from NOP to LAL July 6, 2019. 2020-21 i...",2013,27093019,No,7.926291e+07,5.21699e+07,8
459,LeBron James,35,2020,LAL,67,8.4,6.1,1,2003,37436858,Cap Space,"Signed 4-yr maximum salary contract on July 9,...",2004,37436858,No,8.953773e+07,5.21009e+07,17
142,Bam Adebayo,22,2020,MIA,72,3.4,3.3,14,2017,3454080,1st Round Pick,"Team exercised 2020-21 option on September 11,...",2018,3454080,No,4.843844e+07,4.49844e+07,3
447,Kawhi Leonard,28,2020,LAC,57,8.9,5.1,15,2011,32742000,Cap Space,2021-22 is player option. Signed 3-yr maximum-...,2012,32742000,No,7.485941e+07,4.21174e+07,9
141,Jayson Tatum,21,2020,BOS,66,4,3.4,3,2017,7830000,1st Round Pick,"2020-21 team option exercised October 14, 2019...",2018,7830000,No,4.990628e+07,4.20763e+07,3


## NET & CAV, Visualized

Any surprises here? 

In [164]:
cost_per_vorp

14678316.337096147

In [57]:
%%HTML
<div class='tableauPlaceholder' id='viz1602124752646' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;NB&#47;NBATeamNET&#47;Dashboard1&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='NBATeamNET&#47;Dashboard1' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;NB&#47;NBATeamNET&#47;Dashboard1&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1602124752646');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='1000px';vizElement.style.height='827px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='1000px';vizElement.style.height='827px';} else { vizElement.style.width='100%';vizElement.style.height='927px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

Cumutively, the team NETs show that the Bucks, Clippers, and Celtics have the highest value teams.  These teams have superstars, however, and I think that it would be valuable to dig deeper and see the median NETs of each team and see how balanced they are across their roster.   

In [99]:
%%HTML
<div class='tableauPlaceholder' id='viz1603072078644' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;NB&#47;NBATopNETValuePlayers2019-20RegularSeason&#47;Players&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='NBATopNETValuePlayers2019-20RegularSeason&#47;Players' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;NB&#47;NBATopNETValuePlayers2019-20RegularSeason&#47;Players&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1603072078644');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

I think NET really excels here.  Aside from the max contract perennial superstars whose value is known (Lebron James, James Harden, Kahwi Leonard), NET highlights players who we "know" are good (yet their salaries do not reflect it yet) who are giving an amazing return on their salary.  On a rookie deal, Luka Doncic has the most valuable NET in the league. Other players on rookie deals - Domantas Sabonis, Trae Young, Mitchell Robinson, and Bam Adebayo are among the top 20 NET players in the league! In addition, NET finds value in players who are neither on a rookie deal nor yet max players (Fred VanVleet, Montrezl Harrell).  

One key takeaway I have here is that the value of getting above average players on rookie deals is incredibly high due to the salary structure of these contracts under the current CBA. Which really places a premium on draft picks and the value those draft picks hold in trades.  

This idea of the higher NET value on rookie deals is fleshed out a bit more below:

In [165]:
%%HTML
<div class='tableauPlaceholder' id='viz1603214358148' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;NB&#47;NBANETValuesBasedonSeasonNumber&#47;Sheet7&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='NBANETValuesBasedonSeasonNumber&#47;Sheet7' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;NB&#47;NBANETValuesBasedonSeasonNumber&#47;Sheet7&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1603214358148');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

We can see how the cumutalive NET value peaks through seasons 2-4 before starting a general downward trend.  As we can see below and as we could have guessed - median player contract value goes down as seasons and salaries rise:  

In [167]:
%%HTML
<div class='tableauPlaceholder' id='viz1603216660188' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;NB&#47;NBAMedianNETValuevs_MedianSalary&#47;Median&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='NBAMedianNETValuevs_MedianSalary&#47;Median' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;NB&#47;NBAMedianNETValuevs_MedianSalary&#47;Median&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1603216660188');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

### Future Paths of Analysis 

Now that the basis for player value, CAV, has been defined, there are several possible avenues to go for further analysis: 


1) Grading all transactions based on CAV

With each transaction, there is a net CAV added to each team/GM.  As such, it is possible to grade each transaction and then use that grade in further grouping of transactions.  Within a particular GM (or team, player, division, conference etc), a user would have the ability to group transactions by time period, player position, or transaction type (trade, signing, draft, etc) and also graph/compare their selection to a comparable selection. 

This will allow quantitative answers to questions such as:

  - "What was the most lopsided trade of the past 10 years?"
  - "Who are the top 10 contracts in the NBA?"
  - "Was Phil Jackson a good drafting GM despite his horrible W-L record as Knicks GM?"
  - "How is the Kristaps Porzingus trade graded now vs when it was made?"
  - "What was the worst free agent signing during the last offseason?"


2) Grading drafted players and assigning value to future draft picks

Since there is no way to assign future draft picks a CAV based on statistics, a compilation of historical stats based on draft position can be undertaken.  From these stats, a four year total CAV can be calculated for each draft position, based on the standard first round rookie contracts from the current CBA.  Using this, future draft picks now have a quantitative value which can be used in evaluation of trades involving draft picks.  

Since the exact draft position of a future pick is almost always unknown (unless it is traded between the window of the draft lottery and the draft), this model uses a range to determine where a draft pick might fall.  For example, in 2019-20, the Lakers trading their first round pick during the season would be projected to be in the bottom 20% of the first round, between picks 24-30.  Therefore the CAV for that pick would be the average of the historical CAVs for picks 24-30.  


3) Grading cap flexibility

There is perhaps no quicker fix-all for a GM and a team than free agent signings.  As such, successfully creating, keeping, and maneuvering among cap space remains possibly a GM’s most important attribute other than talent evaluation. 

We can take the average value gained per dollar spent through free agent signings during historical offseasons and scale that value to cap space created or lost during a transaction.  The longest possible contract that can be given in the NBA is five seasons, so this portion of the model could add or take away from a team’s total possible cap space over this time period. 